# はじめての量子ゲーム

このテキストブックをすでにある程度読んでいる場合は、少なくとも量子ゲートを多少は理解している必要があります。学んだ知識をどう活かすか迷っているなら、ゲームを作るとよいでしょう。簡単なゲーム作成は、新しいプログラミングの知識を楽しんで試す手です。

ゲームを作るにはゲームエンジンが必要です。ここではテキストブックのJupyterノートブックで実行できる簡単なゲームエンジンを紹介します。これを使用して単一量子ビットゲートに基づいた非常に単純なゲームを作成します。

まずはゲームエンジンをインポートします。

In [ ]:
from qiskit_textbook.games.qiskit_game_engine import QiskitGameEngine

## 簡単な例

このゲーム エンジンを使用する最小限の例は、単純にすべてのピクセルを特定の色に設定することです。

In [ ]:
# function called when setting up
def start(engine):
    # just move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen.pixel[x,y].set_color('green')
    
# run the game for an 8x8 screen
engine = QiskitGameEngine(start,next_frame,L=8)

次に、特定のピクセルを別の色に設定しましょう。

In [ ]:
# function called when setting up
def start(engine):
    
    # set a parameter to keep track of the player pixel
    engine.X = 1
    engine.Y = 2
    
    # then move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen.pixel[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen.pixel[engine.X,engine.Y].set_color('red')
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

矢印ボタンを使用して移動させます。

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen.pixel[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen.pixel[engine.X,engine.Y].set_color('red')
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

画面の端から離れると、エラーが発生します。これは修正できます。

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen.pixel[x,y].set_color('green')
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen.pixel[Xs,Ys].set_color('red')
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

ここでは `engine.X`座標と`engine.Y`座標は相変わらず画面外に行けますが、ピクセルは折り返されるように表示されます。これは、ピクセルが別の画面に移動していると解釈できます。

それでは、ピクセルでエキサイティングな世界を探検出来るようにしましょう。関数を使用して、各点の色を決定します。あまり変わらないところから始めましょう。

In [ ]:
import numpy as np

def get_color(X,Y):
    return 'green'

# this is the function that does everything
def next_frame (engine):
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            # get the 'world' coordinates X,Y from the onscreen coordinates x,y
            X = np.floor(engine.X/engine.L)*engine.L+x
            Y = np.floor(engine.Y/engine.L)*engine.L+y
            # set it to whatever colour it should be
            engine.screen.pixel[x,y].set_color(get_color(X,Y))
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen.pixel[Xs,Ys].set_color('red')
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

次に、 `get_color`を変更してビーチを作成しましょう。

In [ ]:
def get_color(X,Y):
    if X<12:
        color = 'green'
    else:
        color = 'orange'
    return color
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

このジェネレーター内に単一量子ビット回路を配置して地形を作成することで単一量子ビットゲートを実験できるようにします。そのためには、いくつかのQiskitツールが必要です。

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

まず、回路の結果がどのようになるかを見てみましょう。これは、特定の回転角に対して`ry`ゲートだけを使用した例です。

In [ ]:
theta = np.pi/2

qc = QuantumCircuit(1)
qc.ry(theta,0)

state = Statevector.from_instruction(qc)
probs = state.probabilities_dict()

print(probs)

ここで、角度$\pi/2$は、この回路の最後での測定結果が`0`または`1`となる可能性が等しいことを意味します。他の角度については、どちらか一方にバイアスをかけることができます。これは、第1章で説明したように、まさにこれらのゲートに期待される動作です。

出力「1」の確率を高さとして使用し、それに応じて色を設定しましょう。

In [ ]:
def get_color(X,Y):
    
    # set an angle for ry based on the coordinate
    theta = X*(np.pi/16)
    
    # create and simulate the circuit
    qc = QuantumCircuit(1)
    qc.ry(theta,0)
    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    
    # get the prob of '1'
    if '1' in probs:
        height = probs['1']
    else:
        height = 0
    
    # set colour accordingly
    if height<0.1: # sea/river
        color = 'blue'
    elif height<0.3: # beach
        color = 'orange'
    elif height<0.9: # grass
        color = 'green'
    else: # mountain
        color = 'grey'
        
    return color
    
# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

より複雑な回転ゲートを追加し、これらのゲートがどのように組み合わされるかを考えることで、より複雑な地形を作成できます。

In [ ]:
import random

seed = [random.random() for _ in range(4)]

def get_color(X,Y):
    
    qc = QuantumCircuit(1)

    theta1 = (seed[0]*X+seed[1]*Y)*np.pi/16
    theta2 = (seed[2]*X-seed[3]*Y)*np.pi/16
    qc.ry(theta1,0)
    qc.rx(theta2,0)

    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    
    try:
        height = probs['1']
    except:
        height = 0
        
    # set colour accordingly
    if height<0.1: # sea/river
        color = 'blue'
    elif height<0.3: # beach
        color = 'orange'
    elif height<0.9: # grass
        color = 'green'
    else: # mountain
        color = 'grey'
        
    return color

# run the game
engine = QiskitGameEngine(start,next_frame,L=8)

次に、自分の回路を作成してより興味深い地形を生成してみて、それを使用して単純な量子ゲートで何ができるかを調べます。または、全く新しいゲームを作ってください！

## ゲームエンジンの使い方

ゲームエンジンを最大限に活用するには、全ての動作の詳細が必要になります。これにより、Dパッドと5つのボタンで制御される、低ピクセル画面（デフォルトでは8x8）用のゲームを作成できます。画面はボタンが押されたときにのみ更新されることに注意してください。

ゲームエンジンはすべて`QiskitGameEngine`オブジェクトに基づいています。これは好きなように命名できます。以下では、これを`engine`と呼びます。

### スクリーン

ピクセルは `engine.screen`を使用して関数で位置指定できます。位置（x、y）のピクセルは、 `engine.screen.pixel[x,y]`として指定されます。 以下の3つの方法があります。

- `set_color(color)` -引数`color`は文字列です： `'grey'` 、 `'green'` 、 `'blue'` 、 `'orange'`または`'red'`のいずれか。
- `set_brightness(bright)` -引数`bright`はブール値です。dimの場合は`False` 、brightの場合は`True`です。
- `set_text(text)` -引数は表示されるテキストです。

ほとんどのピクセルには、数文字以上の文字が含まれないことに注意してください。長いテキストは画面下部の長いピクセルに書き込むことができます。このピクセルには、 `engine.screen.pixel['text']`でアクセスできます。

### コントローラー

コントローラーは `engine.controller`で使用します。ボタンはキー`'down'` 、 `'up'` 、 `'left'` 、 `'right'` 、 `'A'` 、 `'B'` 、 `'X'` 、 `'Y'` 、および`'next'`を使用して位置指定されます。それぞれがJupyterウィジェットボタンオブジェクトです。これらのボタンのいずれかを押すと `next_frame`関数が実行されます。各ボタンの`value`属性を使用してボタンが押されたか（ `True` ）押されていないか（ `False` ）を判別します。

### ゲームループ

ゲームは `start`と`next_frame`の2つの関数を定義することによって作成されます。 `start`関数はゲームの開始時に実行され、 `next_frame`は、ボタンを押してゲームを進めるたびに実行されます。どちらにも、クラス`engine`という1つの引数が必要です。ゲームに必要なすべてのパラメーターは、 `engine`クラスの属性として定義する必要があります。

### まとめ

ゲームは、`start`関数と`next_frame`関数を使用して`QiskitGameEngine`オブジェクトを開始することによって開始されます。キーワード引数`L`を使用して、デフォルトの8x8グリッド以外のサイズを選択することもできます。

```
QiskitGameEngine(start,next_frame,L=8)
```

グリッドサイズは`start`および`next_frame`関数で`engine.L`としてアクセスできることに注意してください。 キーワード引数`L`を使用して、デフォルトの8x8グリッド以外のサイズを選択することもできます。